# Tổng hợp code từ sources:
1. https://www.kaggle.com/dschettler8845/visual-in-depth-eda-vinbigdata-competition-data
2. https://www.kaggle.com/sakuraandblackcat/chest-x-ray-knowledges-for-the-14-abnormalities

## Tổng quan

Dữ liệu được lưu dưới định dạng DICOM

Tập train với 15,000 ảnh

Tập public test với 3,000 ảnh

Ảnh được gắn với 14 nhãn:

0 - Aortic enlargement

1 - Atelectasis

2 - Calcification

3 - Cardiomegaly

4 - Consolidation

5 - ILD

6 - Infiltration

7 - Lung Opacity

8 - Nodule/Mass

9 - Other lesion

10 - Pleural effusion

11 - Pleural thickening

12 - Pneumothorax

13 - Pulmonary fibrosis

14 - "No finding" observation was intended to capture the absence of all findings above

**Ở đây nhãn được gắn bởi nhiều chuyên gia X quang, nghĩa là một ảnh sẽ có nhiều ground-truth!**

### Dữ liệu mẫu

Mô tả dữ liệu:

train.csv: metadata của tập train, với mỗi hàng là một mẫu ảnh, bao gồm tên nhãn và bounding box (một ảnh có thể có nhiều hàng)

![figures](figures/overview_data.PNG)

Các cột trong train.csv

- image_id: tên dữ liệu
- class_name: tên của nhãn có trong ảnh (nêu không có thì hiện "No finding")
- class_id: ID của nhãn có trong ảnh (từ 0 đến 14)
- rad_id: ID của chuyên gia X quang, người gắn nhãn
- x_min: minimum X coordinate of the object's bounding box
- y_min: minimum Y coordinate of the object's bounding box
- x_max: maximum X coordinate of the object's bounding box
- y_max: maximum Y coordinate of the object's bounding box

sample_submission.csv: sample submission file in the correct format

**Đầu ra của bài toán là một model có thể khoanh vùng và phân loại 14 loại bất thưởng trong ảnh**

Tìm hiểu sâu về 14 loại bất thường có thể tham khảo repo sau:

[https://www.kaggle.com/sakuraandblackcat/chest-x-ray-knowledges-for-the-14-abnormalities](https://www.kaggle.com/sakuraandblackcat/chest-x-ray-knowledges-for-the-14-abnormalities)


## Visualize dữ liệu

#### Import libraries

In [1]:
import os
from tqdm import tqdm
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

#### Setup global variables

In [8]:
DATA_DIR = "E:/Kaggle/VinBigData"

# Define the paths to the training and testing dicom folders respectively
TRAIN_DIR = os.path.join(DATA_DIR, "train")
TEST_DIR = os.path.join(DATA_DIR, "test")

In [9]:
# Capture all the relevant full train/test paths
TRAIN_DICOM_PATHS = [os.path.join(TRAIN_DIR, f_name) for f_name in os.listdir(TRAIN_DIR)]
TEST_DICOM_PATHS = [os.path.join(TEST_DIR, f_name) for f_name in os.listdir(TEST_DIR)]
print(f"\n... The number of training files is {len(TRAIN_DICOM_PATHS)} ...")
print(f"... The number of testing files is {len(TEST_DICOM_PATHS)} ...")


... The number of training files is 15000 ...
... The number of testing files is 3000 ...


In [10]:
# Define paths to the relevant csv files
TRAIN_CSV = os.path.join(DATA_DIR, "train.csv")
SS_CSV = os.path.join(DATA_DIR, "sample_submission.csv")


In [14]:
# Create the relevant dataframe objects
train_df = pd.read_csv(TRAIN_CSV)
ss_df = pd.read_csv(SS_CSV)

In [15]:
print("\n\nTRAIN DATAFRAME\n\n")
display(train_df.head(3))



TRAIN DATAFRAME




,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0


In [16]:
print("\n\nSAMPLE SUBMISSION DATAFRAME\n\n")
display(ss_df.head(3))



SAMPLE SUBMISSION DATAFRAME




,image_id,PredictionString
0,002a34c58c5b758217ed1f584ccbcfe9,14 1 0 0 1 1
1,004f33259ee4aef671c2b95d54e4be68,14 1 0 0 1 1
2,008bdde2af2462e86fd373a445d0f4cd,14 1 0 0 1 1
